# Note that this project is coded for R

![Example grocery store](grocery_store.jpg)

## 📖 The Background
In the bustling world of retail, especially in grocery stores, understanding consumer preferences and shopping habits is pivotal for business optimization. Imagine a local grocery chain seeking to enhance customer satisfaction and drive sales. By analyzing transactional data from two of its 24-hour stores, each with unique data storage practices due to the store owners' diverse backgrounds, the chain aims to uncover hidden patterns in consumer behavior. The goal here is multifaceted: optimizing store layouts, tailoring marketing strategies, and efficiently managing inventory to align with customer preferences. For instance, if analysis reveals that customers frequently purchase certain products together or show a preference for shopping at specific hours, the store can adjust its marketing efforts and stock levels accordingly. Additionally, understanding seasonal trends and weekly fluctuations in sales helps in planning promotions and staffing. Acting as the retail data scientist for this chain, you'll leverage detailed retail data to drive strategic decisions, ultimately enhancing customer experience and boosting the store's performance. 

## 💾 The Data
`grocery_data1.csv` and `grocery_data2.csv` contain detailed records of grocery transactions from these two stores with the same column names:

| Variable         | Description                                           |
|------------------|-------------------------------------------------------|
| `CustomerID`     | Unique identifier for each customer                   |
| `DateRaw`        | Raw date of the transaction                           |
| `Time`           | Time of the transaction                               |
| `TransactionID`  | Unique identifier for each transaction                |
| `ProductName`    | Name of the product purchased                         |
| `PriceUSD`       | Price of the product in US dollars                    |
| `Quantity`       | Number of the product purchased                     |
| `PaymentMethod`  | Payment type used for the transaction               |
| `Category`       | Category of the product                               |


# Objectives
1) In order to understand consumer spending patterns, what week of the year during the time period given (June 1 to August 31) had the smallest absolute deviation in sales value compared to the mean weekly sales over that same time period?
2) In order to tailor marketing strategies, what hour of the day (as a number on the 24 hour scale) had the most hourly total sales?
3) In order to evaluate individual buying preferences, how many days went by between the three purchases of cornflakes by `CustomerID` 107?

In [172]:
# Import packages
library(dplyr)
library(lubridate)
library(readr)


In [173]:
#import the data
store1 = read_csv('grocery_data1.csv')
    #2,619 rows
store2 = read_csv('grocery_data2.csv')
    #2,581 rows

#investigate the data
head(store1)
#head(store2)

Rows: 2619 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DateRaw, ProductName, PaymentMethod, Category
dbl  (4): CustomerID, TransactionID, PriceUSD, Quantity
time (1): Time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2581 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DateRaw, ProductName, PaymentMethod, Category
dbl  (4): CustomerID, TransactionID, PriceUSD, Quantity
time (1): Time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CustomerID,DateRaw,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category
<dbl>,<chr>,<time>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>
41,"June 28, 2023",20:00:00,2,Apples,5.64,5,Cash,Produce
170,"August 18, 2023",06:00:00,3,Apples,17.92,1,Mobile Payment,Produce
86,"August 18, 2023",09:00:00,4,Pasta,19.14,2,Mobile Payment,Grains
178,"August 06, 2023",02:00:00,5,Rice,0.76,4,Debit Card,Grains
87,"July 30, 2023",09:00:00,6,Chickpeas,11.30,1,Debit Card,Vegetarian
12,"June 17, 2023",15:00:00,8,Onions,11.04,2,Debit Card,Produce


The 'DateRaw' variable has character types. Analysis would benefit if this variable were converted to a date type. In the first dataset, the values are formatted as "_Month_ DD, YYYY". For the second, there is no comma; "DD _Month_ YYYY".  
Both datasets have observations between 2023-06-01 through 2023-08-31.

Moreover, it would be useful to have a 'Sales' quantity which can be built by multiplying the 'PriceUSD' & 'Quantity' variables.

Another variable was created that merged the two datasets together.

In [174]:
#convert 'DateRaw' to date types
store1$Date <- as.Date(store1$DateRaw, format="%B %d, %Y")
store2$Date <- as.Date(store2$DateRaw, format="%d %B %Y")

#calculate Sales
store1$Sales <- store1$PriceUSD * store1$Quantity
store2$Sales <- store2$PriceUSD * store2$Quantity

#merge the datasets
merged_df <- bind_rows(store1, store2)
    #5,200 rows
glimpse(merged_df)

Rows: 5,200
Columns: 11
$ CustomerID    <dbl> 41, 170, 86, 178, 87, 12, 104, 140, 104, 78, 47, 143, 88…
$ DateRaw       <chr> "June 28, 2023", "August 18, 2023", "August 18, 2023", "…
$ Time          <time> 20:00:00, 06:00:00, 09:00:00, 02:00:00, 09:00:00, 15:00…
$ TransactionID <dbl> 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 15, 16, 17, 19, 21, 25,…
$ ProductName   <chr> "Apples", "Apples", "Pasta", "Rice", "Chickpeas", "Onion…
$ PriceUSD      <dbl> 5.64, 17.92, 19.14, 0.76, 11.30, 11.04, 12.04, 4.29, 11.…
$ Quantity      <dbl> 5, 1, 2, 4, 1, 2, 2, 2, 1, 1, 4, 5, 2, 5, 4, 4, 3, 5, 2,…
$ PaymentMethod <chr> "Cash", "Mobile Payment", "Mobile Payment", "Debit Card"…
$ Category      <chr> "Produce", "Produce", "Grains", "Grains", "Vegetarian", …
$ Date          <date> 2023-06-28, 2023-08-18, 2023-08-18, 2023-08-06, 2023-07…
$ Sales         <dbl> 28.20, 17.92, 38.28, 3.04, 11.30, 22.08, 24.08, 8.58, 11…


# Q1
In order to understand consumer spending patterns, what week of the year during the time period given (June 1 to August 31) had the smallest absolute deviation in sales value compared to the mean weekly sales over that same time period?  
Enter your answer into the `smallest_sales_deviation` object as an integer.

- This question basically asks for the (absolute) difference between the total sales each week & the overall weekly average in total sales.

In [175]:
#Create a new variable indicating the week number of a date
merged_df$Week <- week(merged_df$Date)
    #Weeks span 22-35

#Need to calculate the avg weekly sales overall. First, calculate total sales per week.
avg_wk_sales <- merged_df %>%
    group_by(Week) %>%
    summarize(tot_wk_sales = sum(Sales)) %>%
    ungroup()
#Retrieve the avg weekly sales overall; ~$11,443
ov_avg_wk_sales <- mean(avg_wk_sales$tot_wk_sales)

#Find the absolute difference between the overall avg weekly sales quantity & avg sales per week
avg_wk_sales <- avg_wk_sales %>%
    group_by(Week) %>%
    mutate(avg_wk_diff = abs(ov_avg_wk_sales - tot_wk_sales)) %>%
    ungroup()

#Obtain the week with the smallest absolute deviation; 24
smallest_sales_deviation <- avg_wk_sales %>%
    slice_min(avg_wk_diff, n=1) %>%
    pull(Week)
smallest_sales_deviation

[1] 24

# Q2
In order to tailor marketing strategies, what hour of the day (as a number on the 24 hour scale) had the most hourly total sales?  
Enter your answer into the `most_hourly_sales` object as an integer.

- To retrieve the hour of a variable with times, can use `hour()`.

In [176]:
#Calculate the total sales per hour
tot_hr_sales <- merged_df %>%
    group_by(hour = hour(Time)) %>%
    summarize(tot_sales = sum(Sales)) %>%
    ungroup()

#Retrieve the hour with the most total sales; 22
most_hourly_sales <- tot_hr_sales %>%
    slice_max(tot_sales, n=1) %>%
    pull(hour)
most_hourly_sales

[1] 22

# Q3
In order to evaluate individual buying preferences, how many days went by between the three purchases of cornflakes by `CustomerID` 107?  
Enter your answer into the `cornflakes_days` object as an integer vector in the format `c(num1, num2)` where `num1` is the number of days between the first purchase and the second and `num2` for the second and third purchases.

In [177]:
#Filter for the relevant data
q3_df <- merged_df %>%
    filter(CustomerID == 107 & ProductName == 'Cornflakes') %>%
    #sort by date
    arrange(Date) %>%
    select(CustomerID, ProductName, Date) %>%
    #Create a variable calculating the number of days between consecutive purchases
    mutate(days_between_purch = as.numeric(Date - lag(Date, n=1)))
#Retrieve the two date differences & assign to the specified variable
cornflakes_days <- q3_df %>%
    slice(2:3) %>%
    pull(days_between_purch)
cornflakes_days

[1]  6 40